# setting up 

In [1]:
import langchain
import os

llm_model = "gpt-3.5-turbo"

openai_api_key = os.getenv("OPENAI_API_KEY")

import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.chat_models import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model,openai_api_key=openai_api_key)


c:\Users\Mekki\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Reading all txt files from KC-LLM-data

In [2]:
file = open("./KC-LLM-data/Wiki-kc.txt","r",encoding="utf-8")

# Transforming them into data set

the needed data is made from this columns

Instruction:
{instruction that describes a task}

Input:
{input that provides further context}

Response:
{response that appropriately completes the request}

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional
import re

# Define the structure of the output using Pydantic models

class Element(BaseModel):
    instruction: str = Field(..., description="The question about the information.")
    input: Optional[str] = Field(None, description="Additional information to help build the answer.")
    response: str = Field(..., description="The information that was asked for.")

class ReviewOutput(BaseModel):
    elements: List[Element] = Field(..., description="List of elements extracted from the text.")


def parse_response_text(response_text: str) -> ReviewOutput:
    elements = []
    element_pattern = r"élément\s*:\s*Instruction\s*:\s*(.*?)\s*Input\s*:\s*(.*?)\s*Réponse\s*:\s*(.*?)\s*(?=élément|$)"
    
    matches = re.findall(element_pattern, response_text, re.DOTALL)
    
    for match in matches:
        instruction = match[0].strip()
        input_data = match[1].strip() if match[1].strip().lower() != "null" else None
        response = match[2].strip()
        elements.append(Element(instruction=instruction, input=input_data, response=response))
    
    return ReviewOutput(elements=elements)

In [3]:
review_template = """\
For the following text, extract the following information that will be used to train a language model:\
return me a list of elements to get information from the text in the following form all in french\
avoid repeating questions and extract as many as possible\
element :
    Instruction:question about the text\

    Input: Additional information given with the instruction to help build the answer like names,dates,..etc can be null if no more info is needed to answer\

    Response: the information that was asked for\

text: {text}
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)


In [ ]:
source="""League of Legends
    2020 : Kameto Corp et la Division 2
    Le 30 mars 2020, Kameto annonce la création de sa structure d'e-sport, nommée Kameto Corp. Ce dernier a racheté la place de la structure suisse d'Unfazed e-sport en pleine saison de Division 2 Française, ainsi que les joueurs de la structure pour continuer la saison. Adam « Adam » Maanane, Philippe « Akabane » Le, Ilias « Nuclearint » Bizriken, Julien « jujutw0 » Izzo et Jérémy « Helaz » François sont les cinq premiers joueurs de la structure.

    2021 : Débuts en LFL et doublé européen historique
    Le 17 novembre 2020, lors d'un live sur la chaine Twitch de Kameto, les dirigeants Kamel et Prime annoncent l'arrivée de leur structure e-sport en LFL (la première division française). C'est à la suite de ce live que la Kameto Corp est renommée Karmine Corp et change légèrement son logo. La Karmine annonce aussi sa première recrue pour le segment de printemps de LFL 2021 : le toplaner Adam « Adam » Maanane, qui sera le seul élément conservé du parcours de Division 2.

    Le 23 décembre 2020, la Karmine annonce le reste de leur équipe avec l'arrivée du polonais Jakub « Cinkrof » Rokicki évoluant au poste de jungler, l'ancien joueur de la Team Vitality Lucas « Saken » Fayard évoluant au poste de mid laner, l'anglais Matthew « xMatty » Coombs en AD Carry et le belge Raphaël «Targamas » Crabbé, déjà champion de France l'année dernière avec les Misfits Premier, comme support. Déjà présent dans le staff l'année dernière, c'est Yanis « Striker » Kella qui sera le coach de l'équipe.

    La Karmine Corp n'était pas donnée favorite pour remporter le championnat avant le début de la saison et leurs deux premières semaines allaient dans ce sens, avec trois victoires pour deux défaites. La Karmine s'adjugea cependant le derby contre Solary avec des audiences records pour un match de championnat régional. À partir de février, l'équipe monta en puissance et ne perdra que 3 rencontres (dont le match retour face à Solary), finissant en tête du classement de la saison régulière à égalité avec Misfits Premier, l'équipe académique des Misfits Gaming. Le tie-break pour la première place entre les deux équipes fut adjugé par la Karmine Corp qui se qualifia ainsi directement pour la finale du championNationalité Une finale qu'elle remportera facilement 3-0 toujours contre les lapins, avec notamment une prestation de haute volée de son toplaner Adam.

    Cette victoire leur permettra de se qualifier pour les European Masters, la seconde plus grande compétition européenne et le trophée le plus prestigieux que la Karmine Corp puisse remporter. Directement engagée en phase de groupes, la Karmine survole son sujet en battant aisément tous ses adversaires, notamment son ancien joueur Nuclearint, parti chez l'équipe réserve de Schalke 04. Dans l'arbre final et dans un format en deux manches gagnantes qu'ils n'apprécient pas, la Karmine a été bousculée face à l'équipe serbe des ŠAIM SE SuppUp en quarts (la Karmine concédant la première manche avant de battre les Serbes dans les deux matchs suivants) et face à l'UCAM Esports Club en demi-finale (victoire 2-1 de la Karmine Corp dans une rencontre très serrée, débloquée en toute fin de partie par leur midlaner Saken.) En finale, la Karmine Corp retrouve l'un de ses adversaires de la phase de groupe, les Anglais de BT Excel, équipe réserve d'Excel Esports. Dans une rencontre plus accrochée qu'en groupe, la Karmine Corp triomphe 3-1 des Anglais et remporte le trophée, devenant la première équipe française à remporter la ligue régionale et les European Masters pendant le même segment. Cette victoire vaudra notamment les félicitations du président de la République Emmanuel Macron par message privé sur Twitter.

    La Karmine Corp annonce le départ de son jeune toplaner Adam le 12 mai 2021. Il rejoint quelques jours plus tard l'équipe de Fnatic, qui joue en LEC, la plus haute division européenne. Le 29 mai 2021, Kamel et Amine annoncent que Lucas « Cabochard » Simon-Meslet remplacera Adam pour la seconde partie de saison. Cabochard est un vétéran de la scène League of Legends et est l'ancien toplaner de la Team Vitality, avec qui il a disputé les championnats du monde en 2018.

    Après une première moitié de segment impériale (9 victoires en 9 matchs), la Karmine éprouve davantage de difficultés par la suite mais parvient à finir de nouveau premiers de la saison régulière, leur permettant d'accéder directement à la finale du segment ainsi qu'à la phase de groupe des European Masters dont ils sont tenants du titre. Ils retrouvent en finale les Misfits Premier dans un remake de la finale du printemps. Cependant, c'est les lapins qui prennent leur revanche en battant la Karmine Corp 3-1. Cette défaite est la première pour la Karmine Corp dans une rencontre en plus d'un match gagnant, ainsi que le premier titre qui échappe à la Karmine en 2021.

    Juste avant le début des European Masters, la Karmine Corp renverse Vitality.Bee 3-2 dans le cadre de la Coupe de France. Rapidement menée 2-0, la Karmine remporte les trois manches suivantes pour se qualifier in extremis pour la finale à Monaco et prendre leur revanche sur Misfits Premier. Ils se rassurent également juste avant le début de la compétition européenne.

    Les European Masters confirment la forme retrouvée de la Karmine Corp, qui domine facilement sa poule puis le UCAM Esports Club en quarts-de-finale, que la Karmine bat 3-0. Ils affrontent alors Berlin International Gaming qui a recruté l'ancien jungler de la Kameto Corp en Division 2, Akabane. Dans une rencontre serrée qui va jusqu'à la cinquième manche, l'équipe française finit par dominer les Allemands pour se qualifier. Alors que les observateurs attendaient une nouvelle revanche entre la Karmine Corp et Misfits Premier en finale, l'équipe académique de Fnatic, les Fnatic Rising, joue les trouble-fêtes en éliminant les lapins en demi-finale. Dans une nouvelle finale franco-anglaise, la Karmine Corp prend rapidement le large 2-0 avant de se faire remonter à 2-2. Comme contre BIG, la Karmine Corp balaye son adversaire dans la cinquième manche et remporte ainsi ses seconds European Masters. Jamais une équipe n'avait réussi à conserver son titre dans cette compétition.

    Pour son dernier match de l'année, en finale de la Coupe de France, la Karmine Corp joue la belle contre son rival Misfits Premier. Dans une rencontre accrochée, la KCorp finissent par renverser la Karmine Corp, qui termine sa saison sur une défaite 3-2, et laissent les lapins repartir avec le titre de meilleure équipe de France en 2021.


    Liste détaillée des matchs de la Karmine Corp lors de la saison 2021
    
    2022 : Deuxième saison LFL
    Le 15 novembre 2021, à l'occasion d'une conférence de presse diffusée sur Twitch, Kameto et Prime annoncent le départ de Cinkrof, Targamas et xMatty de l'équipe, ainsi que la prolongation du staff technique composé de Striker et Reha jusqu'en 2023. Quelques heures plus tard, à l'ouverture du mercato, la Karmine Corp annonce sur Twitter l'arrivée du AD Carry suédois Martin « Rekkles » Larsson, en provenance de chez G2 Esports. Il était sur la liste des transferts après la non-qualification aux Worlds 2021 de G2, malgré une équipe taillée pour le titre suprême. Pour s'offrir un joueur de ce calibre, le transfert aurait coûté près d'un million d'euros à la Karmine. Le roster est ensuite complété par la prolongation d'un an de Cabochard, le maintien de Saken à son poste, ainsi que par les arrivées libres de Jules « Hantera » Bourgeois, du jeune Doğukan « 113 » Balcı (aussi surnommé Bumm) et de Wao « Wao » Dai comme joueur remplaçant.

    La Karmine Corp dispute son premier match de la saison à l'occasion de la présentation de la nouvelle équipe e-sport KOI à Barcelone, fondée par le streamer espagnol Ibai Llanos et le footballeur Gérard Piqué. Dans un showmatch en 2 manches gagnantes, la Karmine perd 2-1, en ayant dominé lors des trois matchs le début de partie, mais en se faisant reprendre par deux fois à cause de combats mal négociés. Cette rencontre a suscité une énorme attention avec un pic à 477 000 spectateurs sur les streams d'Ibai et de Kameto, en plus des 15 000 spectateurs présents au Palau Sant Jordi, dont 300 ultras Karmine. Le match retour a lieu le 8 Janvier 2022 au Carrousel du Louvre à Paris. Devant un public d'un millier de personnes, la Karmine prend sa revanche sur les KOI en les battant 2-1. La rencontre a de nouveau suscité une grande attention avec un pic à 366 000 spectateurs. La Karmine a également officialisé durant l'événement l'arrivée d'Orange comme sponsor principal maillot.

    Dans une LFL plus forte que jamais, la Karmine Corp démarre mal le championnat, perdant quatre de ses six premiers matchs. Dos au mur, la Karmine Corp se réveille en 4e semaine, jouée à l'Acropolis de Nice devant le public. Avec deux victoires écrasantes contre GameWard avec un Pentakill (cinq éliminations en une séquence de jeu) de Rekkles, qui tient enfin sa partie de référence, ainsi que Vitality.Bee jusque-là impérial. L'équipe revigorée entame alors une remontée du classement, et terminera deuxième de la saison régulière, à une victoire des LDLC OL.

    Toute cette dynamique positive vole cependant en éclats durant les play-offs de LFL. Tout d'abord balayés sèchement par LDLC OL 3-0 (jamais la Karmine n'avait perdu de BO5 (best of 5, le premier qui gagne trois des cinq parties gagne le match) sur ce score) dans la demi-finale des gagnants, ils sont battus 3-1 par Team BDS Academy (qui remplace Team BDS parti depuis en LEC) dans la demi-finale des perdants et terminent 3e de cette saison de LFL. L’équipe est qualifiée aux Amazon European Masters et défendra son titre, mais devra passer par un tour supplémentaire.

    Le tour préliminaire des Amazon European Masters fut sans grande embuche pour la KC, battant tous ses adversaires sèchement en play-in. Malgré un mauvais départ face à X7 Esports en groupe, la KC monte en puissance et finit premier de son groupe en prenant leur revanche contre les Anglais, puis bat l'équipe académique d'Unicorns of Love en quarts-de-finale 3-1. Dans des European Masters complètement dominé par les équipes françaises, la Karmine Corp affronte Vitality.Bee en demi-finale. Comme l'année dernière en Coupe de France, les abeilles remportent les deux premières manches et pousse la KC dans ses retranchements. Et comme l'année dernière, la Karmine Corp renverse la situation et s'impose 3-2. En finale se dresse une autre équipe française : LDLC OL, champions de France et invaincus dans la compétition jusqu'à présent. Après avoir concédé la première manche, la Karmine finit par gagner sur la faille ainsi que sur le plan psychologique, poussant LDLC à commettre des grosses erreurs en draft et en jeu, qui mènera à une défaite 3-1 de LDLC OL et au succès de la Karmine Corp pour ses 3èmes European Masters de suite16.

    Le début du segment d'été se passe parfaitement pour la Karmine Corp, vainqueur à cinq reprises sur six rencontres. Un contexte idéal pour le KCX2. La Karmine joue son match de saison régulière contre Misfits Premier devant plus de 12 000 spectateurs à l'Accor Arena. La fête est entachée d'une défaite et de la fin de sa série de victoires. La Karmine Corp alternera ensuite victoires et défaites sans pour autant parvenir à retrouver son niveau éclatant de la fin du printemps. L'équipe doit attendre la dernière semaine pour se qualifier pour les play-offs de justesse. Ayant terminé 6e de saison régulière, la Karmine n'est pas encore qualifiée pour les Amazon European Masters et devra commencer dans l'arbre des perdants, où la moindre défaite sera éliminatoire. Ils sont alors opposés aux Boulonnais de GameWard.

    Malgré sa saison régulière en deçà des attentes, la KC reste favorite pour les observateurs, s'appuyant sur son expérience dans les rencontres en 3 manches gagnantes et ses individualités supérieures sur le papier. Cependant GameWard domine son adverse sur beaucoup de plans et si la Karmine résiste bien en égalisant à 1-1, l'équipe finit par rompre et s'incline 3-1. Cette défaite a pour conséquence d'éliminer la KC des play-offs de LFL et surtout les empêche de se qualifier aux European Masters, une première depuis l'arrivée de la KC en LFL qui ne pourra pas défendre son titre, signant la fin d'une série historique.


    Liste détaillée des matchs de la Karmine Corp lors de la saison 2022
    
    2023 : Une saison en dents de scie
    Le 9 janvier, la Karmine Corp annonce sa nouvelle équipe pour 2023. Le français Duncan « Skeanz » Marquet évoluant au poste de jungler, le turc Hasan « Kaori » Sentürk évoluant au poste d'ADC et le roumain Alexandru « whiteinn » Kolozsvari évoluant au poste de support rejoignent les français Lucas « Cabochard » Simon-Meslet et Lucas « Saken » Fayard . Stefan « Nerroh » Pereira rejoint Clément « Nalkya » Guillemard et Rehareha « Reha » Ramanana comme entraîneur de l'équipe.

    En parallèle, Arthur Perticoz rejoint Kameto et Prime en tant que directeur de la structure.

    Cette restructuration ne permet pas à l'équipe de performer durant le Winter Split qui finit avec un bilan de 7 victoires pour 11 défaites, ce qui la classe en avant-dernière position de la LFL, synonyme de non-participation aux play-off. En mai 2023, des changements sont effectués en prévision du Summer Split. Raphaël «Targamas » Crabbé et Jakub « Cinkrof » Rokicki tous deux anciens joueurs de l'équipe réintègrent la structure au poste de support et jungler remplaçant respectivement whiteninn et Skeanz. L'ADC Caliste « Caliste » Henry-Hennebert, jeune rookie de 16 ans lors de son recrutement, remplace Kaori. Le coach Reha devient l'entraîneur principale et il est rejoint par Wadi au poste d'analyste.

    Cette restructuration porte ses fruits et permet à l'équipe de prendre la 1re place du classement lors du Summer Split 2023 et de valider leur place en play-offs. La semaine d'après, l'équipe bat BK ROG à l'issue d'un match en 3-1 et se qualifie pour les EMEA Master. Durant ce Summer Split, Caliste bat le record d'éliminations sur une saison régulière et Targamas bat le record d'assistances. Ce record leur vaut d'être appelés par certains « meilleure botlane de l'histoire de la LFL ».

    Le 11 août 2023, la Karmine remporte son deuxième titre de champion de France après un match en 3-0 contre BK ROG.

    Le 1er septembre 2023, la Karmine s'impose 3-2 contre la structure française Team GO lors des demi-finales du Summer Split 2023 des EMEA Master, se qualifiant alors en finale contre l'équipe espagnole Movistar Riders.

    Et c'est alors le 8 septembre que l'équipe Movistar Riders s'incline devant la Karmine Corp, leur laissant la victoire, le match se terminant en 3-2, la Karmine Corp remportant alors le championnat d'Europe 2023. Cette victoire est d'autant plus historique puisque le « midlaner » français Lucas « Saken » Fayard remporte ses 4èmes EMEA Master (en autant de participation) dans une Sud de France Arena survoltée.

    Le 18 octobre 2023, la Karmine Corp annonce son entrée à la LEC et le jeune Caliste prolonge son contrat jusqu'en 2026.

    2024 : Première année en LEC
    La Karmine débute en 2024 dans cette league avec le Winter split et fini dernière avec 2 victoires sur 9, ce qui ne permet pas de se qualifier pour les playoffs.

    Pour le Spring Split, l'équipe arrive une nouvelle fois à la dernière place malgré un changement de coach. Le responsable de la structure, Kameto, a indiqué qu'il y aura du changement au sein des joueurs pour le Summer Split.

    Le premier changement annoncé est pour le post de jungler, Zhou « Bo » Yang-Bo est mis sur le banc et Can « Closer » Çelik est appelé à rejoindre les rangs de la KC. Le toplaner change aussi, Kim « Canna » Chang-Dong est recruté au profit de Lucas « Cabochard » Simon-Meslet. Enfin, Vladimiros « Vladi » Kourtidis sera promu de l'équipe LFL KarmineCorp Blue et prendra la place de Lucas « Saken » Fayard au poste de midlaner.
"""

In [4]:
message = prompt_template.format_messages(text=source)
reponse = chat(message).content
parsed_output = parse_response_text(reponse)


[HumanMessage(content="For the following text, extract the following information that will be used to train a language model:\nreturn me a list of elements to get information from the text in the following form\nelement :\n    Instruction: given the text, what would you ask to get the informations inside of it\n    Input: Additional information given with the instruction to help build the answer\n    Response: the information that was asked for and is contained in the text\ntext: la karmine corp est un club français d'e-sport fondé par kamet0 qui joue principalement a league of legends et valorant\n")]